In [7]:
import win32com.client
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

0


In [9]:
import win32com.client
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
print(instCpStockCode.GetCount())

3381


In [1]:
import win32com.client
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
for i in range(0, 10):
    print(instCpStockCode.GetData(1,i))

동화약품
KR모터스
경방
메리츠화재
삼양홀딩스
삼양홀딩스우
하이트진로
하이트진로2우B
유한양행
유한양행우


In [3]:
#종목정보 가져오기(최초)

import win32com.client
import pandas as pd 

path_data = 'data/'

CPE_MARKET_KIND = {'KOSPI':1,  'KOSDAQ':2}

instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")

rows = list()

for key, value in CPE_MARKET_KIND.items():
    codeList = instCpCodeMgr.GetStockListByMarket(value)
    for code in codeList:
        name = instCpCodeMgr.CodeToName(code)
        sectionKind = instCpCodeMgr.GetStockSectionKind(code)
        row = [code, name, key, sectionKind]
        rows.append(row)

print('모든 종목을 불러왔습니다')

stockitems = pd.DataFrame(data= rows, columns=['code','name', 'section','sectionKind'])

stockitems.loc[stockitems['sectionKind'] == 10, 'section'] =   'ETF'
#stockitems.to_csv(path_data+'stockitems.csv', index=False)

print('파일을 저장하였습니다.')


모든 종목을 불러왔습니다
파일을 저장하였습니다.


In [18]:
#종목정보 가져오기(업데이트)

import win32com.client
import pandas as pd 

path_data = 'data/'

CPE_MARKET_KIND = {'KOSPI':1,  'KOSDAQ':2}

instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")

rows = list()

for key, value in CPE_MARKET_KIND.items():
    codeList = instCpCodeMgr.GetStockListByMarket(value)
    for code in codeList:
        name = instCpCodeMgr.CodeToName(code)
        sectionKind = instCpCodeMgr.GetStockSectionKind(code)
        row = [code, name, key, sectionKind]
        rows.append(row)

print('모든 종목을 불러왔습니다')

stockitems = pd.DataFrame(data= rows, columns=['code','name', 'section','sectionKind'])

stockitems.loc[stockitems['sectionKind'] == 10, 'section'] =   'ETF'
stockitems.to_csv(path_data+'stockitems.csv', index=False, encoding='cp949')   # 윈도우의 경우 encoding='cp949' 추가

print('파일을 저장하였습니다.')


모든 종목을 불러왔습니다
파일을 저장하였습니다.


In [2]:
##주가정보 가져오기  (최초 생성)

import win32com.client
import pandas as pd 
import time
import datetime

path_data = 'data/'

column_dailychart = ['code', 'section', 'date', 'open', 'high', 'low', 'close',
                     'vol', 'value', 'n_stock', 'agg_price', 'foreign_rate','agency_buy', 'agency_netbuy']

stockitems = pd.read_csv(path_data+'stockitems.csv', encoding='cp949')  # 윈도우의 경우 encoding='cp949' 추가

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

row = list(range(len(column_dailychart)))
rows = list()

instStockChart.SetInputValue(1, ord('1'))
#instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(2, '20171231')
instStockChart.SetInputValue(3, '20170101')
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8, 9, 12, 13, 17, 20, 21))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))

j = 0
    
for idx, stockitem in stockitems.iterrows():

    if j >= 10 :
        break
            
    remain_request_count =  nCpCybos.GetLimitRemainCount(1)
    print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

        while True:
            time.sleep(2)
            remain_request_count =  nCpCybos.GetLimitRemainCount(1)
            if  remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
            print('대기 중...')
            
    instStockChart.SetInputValue(0, stockitem['code'])

    
    # BlockRequest
    instStockChart.BlockRequest()

    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)
    j += 1
    # GetDataValue
    for i in range(numData):
        row[0] =  stockitem['code']
        row[1] = stockitem['section']  # 코스피, 코스닥, ETF 여부
        row[2] = instStockChart.GetDataValue(0, i)  # 날짜
        row[3] = instStockChart.GetDataValue(1, i)  # 시가
        row[4] = instStockChart.GetDataValue(2, i)  # 고가
        row[5] = instStockChart.GetDataValue(3, i)  # 저가
        row[6] = instStockChart.GetDataValue(4, i)  # 종가
        row[7] = instStockChart.GetDataValue(5, i)  # 거래량
        row[8] = instStockChart.GetDataValue(6, i)  # 거래대금
        row[9] = instStockChart.GetDataValue(7, i)  # 상장주식수
        row[10] = instStockChart.GetDataValue(8, i)  # 시가총액
        row[11] = instStockChart.GetDataValue(9, i)  # 외국인 보율비율
        row[12] = instStockChart.GetDataValue(10, i)  # 기관순매수
        row[13] = instStockChart.GetDataValue(11, i)  # 기관누적순매수
        rows.append(list(row))   # 그냥 row를 입력하면 그 전에 입력되었던 row까지 다 현재 값으로 바뀜
    

    
print('데이터를 모두 불러왔습니다.')


# 2년 이상 큰 데이터를 저장할 때는 파일을 쪼개서 저장 후 64bit 버전에서 합쳐서 저장
# dailychart = pd.DataFrame(data = rows, columns= column_dailychart) 
# MemoryError: Unable to allocate 190. MiB for an array with shape (3554329, 14) and data type object
# 32bit로 실행할 경우 큰 데이터를 pandas dataframe으로 옮기는 과정에서 위와같은 에러가 발생

import pickle

unit = 500000

i = 0
for i in range(0, int(len(rows)/unit)):
    with open(path_data+dailychart{0}.dat'.format(i), 'wb') as f:
        pickle.dump(rows[i*unit:(i+1)*unit], f)

if i > 0:
    i += 1
with open(path_data+dailychart{0}.dat'.format(i), 'wb') as f:
    pickle.dump(rows[(i)*unit:len(rows)], f)

# 2년 이내 수준의 데이터를 저장할 때는 바로 pandas dataframe으로 변환 후 저장


#dailychart= pd.DataFrame(data = rows, columns= column_dailychart)
#dailychart[['value' ,'agg_price']] = (dailychart[['value' ,'agg_price']]/1000000).astype(int)
#dailychart.to_csv('dailychart.csv', index=False)

print('모든 데이터를 저장하였습니다.')

FileNotFoundError: [Errno 2] No such file or directory: 'stockitems.csv'

In [6]:
len(rows)

14

In [2]:
import win32com.client
import pandas as pd 

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
remain_request_count =  nCpCybos.GetLimitRemainCount(1)
print(remain_request_count)

0


In [2]:
## 분할저장한 주가정보파일 합치기

import pickle
import pandas as pd

path_data = 'data/'

n_file = 10
data = list()
column_dailychart = ['code', 'section', 'date', 'open', 'high', 'low', 'close',
                     'vol', 'value', 'n_stock', 'agg_price', 'foreign_rate','agency_buy', 'agency_netbuy']

for i in range(0, n_file): 
    with open(path_data+dailychart{0}.dat'.format(i), 'rb') as f:
        tmp_data = pickle.load(f)
    data = data + tmp_data

dailychart = pd.DataFrame(data = data, columns= column_dailychart)
dailychart[['value' ,'agg_price']] = (dailychart[['value' ,'agg_price']]/1000000).astype(int)

dailychart = dailychart.sort_values(by=['code','date'])
dailychart.to_csv(path_data+'dailychart.csv', index=False)

print('데이터를 저장하였습니다.')

데이터를 저장하였습니다.


In [23]:
##주가정보 가져오기  (업데이트)

import win32com.client
import pandas as pd 
import time
import datetime

path_data = 'data/'

stockitems = pd.read_csv(path_data +'stockitems.csv')
dailychart_prev = pd.read_csv(path_data+'dailychart.csv')


instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

column_dailychart = ['code', 'section', 'date', 'open', 'high', 'low', 'close',
                     'vol', 'value', 'n_stock', 'agg_price', 'foreign_rate','agency_buy', 'agency_netbuy']
row = list(range(len(column_dailychart)))
rows = list()

instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(3, str(np.max(dailychart['date'])+1))
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8, 9, 12, 13, 17, 20, 21))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))
    
for idx, stockitem in stockitems.iterrows():
    remain_request_count =  nCpCybos.GetLimitRemainCount(1)
    print(stockitem['code'], stockitem['name'], '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        while True:

            print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')
            time.sleep(2)
            remain_request_count =  nCpCybos.GetLimitRemainCount(1)
            if  remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
        
    instStockChart.SetInputValue(0, stockitem['code'])

    # BlockRequest
    instStockChart.BlockRequest()

    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)

    # GetDataValue
    for i in range(numData):
        row[0] =  stockitem['code']
        row[1] = stockitem['section']  # 코스피, 코스닥, ETF 여부
        row[2] = instStockChart.GetDataValue(0, i)  # 날짜
        row[3] = instStockChart.GetDataValue(1, i)  # 시가
        row[4] = instStockChart.GetDataValue(2, i)  # 고가
        row[5] = instStockChart.GetDataValue(3, i)  # 저가
        row[6] = instStockChart.GetDataValue(4, i)  # 종가
        row[7] = instStockChart.GetDataValue(5, i)  # 거래량
        row[8] = instStockChart.GetDataValue(6, i)  # 거래대금
        row[9] = instStockChart.GetDataValue(7, i)  # 상장주식수
        row[10] = instStockChart.GetDataValue(8, i)  # 시가총액
        row[11] = instStockChart.GetDataValue(9, i)  # 외국인 보율비율
        row[12] = instStockChart.GetDataValue(10, i)  # 기관순매수
        row[13] = instStockChart.GetDataValue(11, i)  # 기관누적순매수
        rows.append(list(row))   # 그냥 row를 입력하면 그 전에 입력되었던 row까지 다 현재 값으로 바뀜
    
print('데이터를 모두 불러왔습니다.')

dailychart = pd.DataFrame(data = rows, columns= column_dailychart)

dailychart[['value' ,'agg_price']] /= 1000000
dailychart[['value' ,'agg_price']] = dailychart[['value' ,'agg_price']].astype(int)

dailychart = dailychart.append(dailychart_prev)
dailychart = dailychart.sort_values(by=['code','date'])

dailychart.to_csv(path_data+'dailychart.csv', index=False)
print('데이터를 모두 불러왔습니다.')

KeyboardInterrupt: 

In [4]:
##지수정보 가져오기  (최초 생성)

import win32com.client
import pandas as pd 
import time
import datetime

path_data = 'data/'

column_dailychart = ['code', 'section', 'date', 'open', 'high', 'low', 'close']

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

row = list(range(len(column_dailychart)))
rows = list()

CPE_MARKET_KIND = {'KOSPI':'U001',  'KOSDAQ':'U201'}


instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, '20140714')
instStockChart.SetInputValue(3, '20140101')
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))


for key, value in CPE_MARKET_KIND.items():

    remain_request_count =  nCpCybos.GetLimitRemainCount(1)
    print(key, value, '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

        while True:
            time.sleep(2)
            remain_request_count =  nCpCybos.GetLimitRemainCount(1)
            if  remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
            print('대기 중...')
            
    instStockChart.SetInputValue(0, value)

    
    # BlockRequest
    instStockChart.BlockRequest()

    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)

    # GetDataValue
    for i in range(numData):
        row[0] =  value
        row[1] = key  # 코스피, 코스닥 여부
        row[2] = instStockChart.GetDataValue(0, i)  # 날짜
        row[3] = instStockChart.GetDataValue(1, i)  # 시가
        row[4] = instStockChart.GetDataValue(2, i)  # 고가
        row[5] = instStockChart.GetDataValue(3, i)  # 저가
        row[6] = instStockChart.GetDataValue(4, i)  # 종가
        rows.append(list(row))   
    
print('데이터를 모두 불러왔습니다.')

dailychart= pd.DataFrame(data = rows, columns= column_dailychart)
dailychart = dailychart.sort_values(['code','date'])
dailychart.to_csv(path_data+'dailychart_index.csv', index=False)

print('모든 데이터를 저장하였습니다.')

KOSPI U001 남은 요청 :  0
남은 요청이 모두 소진되었습니다. 잠시 대기합니다.
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...
대기 중...


KeyboardInterrupt: 

In [ ]:
##지수정보 가져오기  (업데이트)

import win32com.client
import pandas as pd 
import time
import datetime

column_dailychart = ['code', 'section', 'date', 'open', 'high', 'low', 'close']

dailychart_prev = pd.read_csv('dailychart_index.csv')

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

row = list(range(len(column_dailychart)))
rows = list()

CPE_MARKET_KIND = {'KOSPI':'U001',  'KOSDAQ':'U201'}


instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(3, str(np.max(dailychart['date'])+1))
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))

for key, value in CPE_MARKET_KIND.items():

    remain_request_count =  nCpCybos.GetLimitRemainCount(1)
    print(key, value, '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

        while True:
            time.sleep(2)
            remain_request_count =  nCpCybos.GetLimitRemainCount(1)
            if  remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
            print('대기 중...')
            
    instStockChart.SetInputValue(0, value)

    
    # BlockRequest
    instStockChart.BlockRequest()

    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)

    # GetDataValue
    for i in range(numData):
        row[0] =  value
        row[1] = key  # 코스피, 코스닥 여부
        row[2] = instStockChart.GetDataValue(0, i)  # 날짜
        row[3] = instStockChart.GetDataValue(1, i)  # 시가
        row[4] = instStockChart.GetDataValue(2, i)  # 고가
        row[5] = instStockChart.GetDataValue(3, i)  # 저가
        row[6] = instStockChart.GetDataValue(4, i)  # 종가
        rows.append(list(row))  
    
    
print('데이터를 모두 불러왔습니다.')

dailychart= pd.DataFrame(data = rows, columns= column_dailychart)
dailychart = dailychart.append(dailychart_prev)
dailychart = dailychart.sort_values(by=['code','date'])
dailychart.to_csv('dailychart_index.csv', index=False)

print('모든 데이터를 저장하였습니다.')

In [2]:
#ETF 종목 유형별 분류

import win32com.client
import pandas as pd 
import numpy as np
import time

stockitems = pd.read_csv('stockitems(210414).csv')
pd.set_option('display.max_rows', 500)

stockitems.loc[stockitems['sectionKind'] == 10, 'name']
etf = stockitems.loc[stockitems['sectionKind'] == 10]

In [8]:
list_index_keywords = []
list_inverse_keywords = ['인버스']
list_foreign_keywords = ['미국', '일본', '중국', '차이', 'China', '인도', '베트남', '필리핀', '싱가포르', '러시아', '멕시코',
                         '유로', '라틴', '아시아', '선진', '이머징', '신흥국', '글로벌']
list_commodity_keywords = ['원유', '농산물', '콩']
list_metal_keywords = ['금속', '금은', '골드', '은선물', '구리']
list_bond_keywords = ['채']

index = etf.loc[etf['name'].str.contains('|'.join(list_inverse_keywords))].index
etf.loc[index,'inverse'] = 1

index = etf.loc[etf['name'].str.contains('|'.join(list_foreign_keywords))].index
etf.loc[index,'foreign'] = 1

index = etf.loc[etf['name'].str.contains('|'.join(list_commodity_keywords))].index
etf.loc[index,'commodity'] = 1

index = etf.loc[etf['name'].str.contains('|'.join(list_metal_keywords ))].index
etf.loc[index,'metal'] = 1

index = etf.loc[etf['name'].str.contains('|'.join(list_bond_keywords))].index
etf.loc[index,'bond'] = 1

C:\Users\SEUNGU_CHOI\anaconda3\envs\py38_32\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [8]:
import win32com.client

# Create object
instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

# SetInputValue
instStockChart.SetInputValue(0, 'A000020')
instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, '20210415')
instStockChart.SetInputValue(3, '20140101')
#instStockChart.SetInputValue(4, 1800)
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8, 9, 12, 13, 14, 15, 16, 17, 20, 21))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))

# BlockRequest
instStockChart.BlockRequest()

# GetHeaderValue
numData = instStockChart.GetHeaderValue(3)
numField = instStockChart.GetHeaderValue(1)
    
for i in range(numData):
    for j in range(numField):
        print(instStockChart.GetDataValue(j, i), end=" ")
    print("")

20210415 15850 15900 15450 15600 204149 3189000000 27931000 435723000000 27931470 27042562 888908 3.180000066757202 4274 122783 
20210414 15250 15950 15100 15800 433602 6793000000 27931000 441309000000 27931470 27016844 914626 3.2699999809265137 1139 118509 
20210413 15500 15550 15250 15300 186169 2860000000 27931000 427344000000 27931470 27036960 894510 3.200000047683716 3427 117370 
20210412 15150 15550 15050 15500 300002 4608000000 27931000 432930000000 27931470 27014930 916540 3.2799999713897705 3747 113943 
20210409 15000 15150 14700 15050 205650 3084000000 27931000 420361000000 27931470 27040524 890946 3.190000057220459 -9034 110196 
20210408 14350 15150 14350 14950 473907 7054000000 27931000 417568000000 27931470 27034151 897319 3.2100000381469727 1630 119230 
20210407 14200 14450 14100 14400 119400 1709000000 27931000 402206000000 27931470 27056249 875221 3.130000114440918 -660 117600 
20210406 14350 14400 14100 14100 138948 1970000000 27931000 393827000000 27931470 27092546 83

20200520 10100 10200 9990 10050 424923 4273000000 27931000 280706000000 27931470 26482320 1449150 5.190000057220459 13479 301063 
20200519 10500 10500 9910 10050 758107 7675000000 27931000 280706000000 27931470 26509397 1422073 5.090000152587891 -12956 287584 
20200518 10550 10800 10200 10700 622190 6602000000 27931000 298861000000 27931470 26528084 1403386 5.019999980926514 12449 300540 
20200515 10850 10950 10450 10550 680362 7270000000 27931000 294672000000 27931470 26575145 1356325 4.860000133514404 -15325 288091 
20200514 10250 10900 10100 10700 1355080 14400000000 27931000 298861000000 27931470 26522450 1409020 5.039999961853027 15933 303416 
20200513 10650 10650 10300 10350 380200 3958000000 27931000 289085000000 27931470 26529454 1402016 5.019999980926514 -10649 287483 
20200512 10350 10650 10150 10600 617130 6426000000 27931000 296068000000 27931470 26511980 1419490 5.079999923706055 5660 298132 
20200511 10600 10600 10150 10300 469424 4839000000 27931000 287689000000 27931470

20190628 9800 9870 9610 9680 18528 179000000 27931000 270372000000 27931470 26115495 1815975 6.5 219 251605 
20190627 9890 9890 9700 9760 23986 234000000 27931000 272606000000 27931470 26113958 1817512 6.510000228881836 3533 251386 
20190626 9860 9900 9790 9840 12594 124000000 27931000 274841000000 27931470 26106398 1825072 6.53000020980835 723 247853 
20190625 9810 9890 9730 9860 25531 250000000 27931000 275399000000 27931470 26102404 1829066 6.550000190734863 2451 247130 
20190624 9910 9920 9820 9880 19246 190000000 27931000 275958000000 27931470 26091920 1839550 6.590000152587891 508 244679 
20190621 9780 9990 9780 9870 95774 952000000 27931000 275678000000 27931470 26083805 1847665 6.610000133514404 -208 244171 
20190620 9930 9950 9810 9820 20025 198000000 27931000 274282000000 27931470 26079887 1851583 6.630000114440918 6025 244379 
20190619 9900 9960 9730 9900 66758 654000000 27931000 276516000000 27931470 26078145 1853325 6.639999866485596 -185 238354 
20190618 9790 10150 9680 9

20180621 11200 11350 11000 11000 54794 609000000 27931000 307241000000 27931470 25306557 2624913 9.399999618530273 158 135315 
20180620 10800 11250 10800 11200 61718 685000000 27931000 312827000000 27931470 25297891 2633579 9.430000305175781 -2359 135157 
20180619 11350 11450 10750 10850 215289 2385000000 27931000 303051000000 27931470 25298604 2632866 9.430000305175781 -1635 137516 
20180618 11800 11900 11200 11450 155385 1779000000 27931000 319809000000 27931470 25276419 2655051 9.510000228881836 -216 139151 
20180615 12000 12050 11750 11800 71864 852000000 27931000 329585000000 27931470 25253062 2678408 9.59000015258789 -16021 139367 
20180614 11650 12000 11550 12000 165452 1957000000 27931000 335172000000 27931470 25237546 2693924 9.640000343322754 59262 155388 
20180612 11500 11750 11500 11650 58240 677000000 27931000 325396000000 27931470 25230913 2700557 9.670000076293945 349 96126 
20180611 11500 11650 11400 11500 35991 416000000 27931000 321206000000 27931470 25233669 2697801 

20170612 9020 9080 8870 8870 72203 646000000 27931000 247747000000 27931470 25335078 2596392 9.300000190734863 -660 74858 
20170609 9140 9180 9000 9050 124202 1125000000 27931000 252775000000 27931470 25326011 2605459 9.329999923706055 529 75518 
20170608 9080 9210 8970 9130 255999 2323000000 27931000 255010000000 27931470 25317481 2613989 9.359999656677246 -53 74989 
20170607 9150 9160 8990 9090 108208 978000000 27931000 253892000000 27931470 25297711 2633759 9.430000305175781 576 75042 
20170605 8990 9240 8970 9140 177175 1617000000 27931000 255289000000 27931470 25276031 2655439 9.510000228881836 878 74466 
20170602 8940 9020 8870 9000 90581 811000000 27931000 251379000000 27931470 25301118 2630352 9.420000076293945 736 73588 
20170601 9010 9040 8840 8940 89610 801000000 27931000 249703000000 27931470 25290858 2640612 9.449999809265137 -777 72852 
20170531 8830 9090 8830 8990 150658 1354000000 27931000 251099000000 27931470 25264098 2667372 9.550000190734863 -940 73629 
20170530 876

20160720 10050 10200 9970 9970 216117 2174000000 27931000 278472000000 27931470 25301877 2629593 9.40999984741211 -13 16970 
20160719 10150 10200 10000 10050 110304 1112000000 27931000 280706000000 27931470 25296736 2634734 9.430000305175781 -317 16983 
20160718 10150 10200 9970 10100 169020 1702000000 27931000 282103000000 27931470 25284970 2646500 9.470000267028809 -72 17300 
20160715 10400 10400 10000 10100 240215 2447000000 27931000 282103000000 27931470 25295902 2635568 9.4399995803833 -4135 17372 
20160714 10500 10500 10150 10300 153051 1578000000 27931000 287689000000 27931470 25291302 2640168 9.449999809265137 -383 21507 
20160713 10750 10850 10400 10450 227423 2409000000 27931000 291878000000 27931470 25279202 2652268 9.5 -26 21890 
20160712 10650 10750 10350 10550 213586 2246000000 27931000 294672000000 27931470 25278655 2652815 9.5 -3037 21916 
20160711 10750 10850 10550 10650 245693 2619000000 27931000 297465000000 27931470 25280297 2651173 9.489999771118164 5008 24953 
201

In [1]:
import win32com.client

instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = instCpCodeMgr.GetStockListByMarket(1)

for i, code in enumerate(codeList):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    print(i, code, secondCode, name)

0 A000020 1 동화약품
1 A000040 1 KR모터스
2 A000050 1 경방
3 A000060 1 메리츠화재
4 A000070 1 삼양홀딩스
5 A000075 1 삼양홀딩스우
6 A000080 1 하이트진로
7 A000087 1 하이트진로2우B
8 A000100 1 유한양행
9 A000105 1 유한양행우
10 A000120 1 CJ대한통운
11 A000140 1 하이트진로홀딩스
12 A000145 1 하이트진로홀딩스우
13 A000150 1 두산
14 A000155 1 두산우
15 A000157 1 두산2우B
16 A000180 1 성창기업지주
17 A000210 1 DL
18 A000215 1 DL우
19 A000220 1 유유제약
20 A000225 1 유유제약1우
21 A000227 1 유유제약2우B
22 A000230 1 일동홀딩스
23 A000240 1 한국앤컴퍼니
24 A000270 1 기아
25 A000300 1 대유플러스
26 A000320 1 노루홀딩스
27 A000325 1 노루홀딩스우
28 A000370 1 한화손해보험
29 A000390 1 삼화페인트
30 A000400 1 롯데손해보험
31 A000430 1 대원강업
32 A000480 1 조선내화
33 A000490 1 대동공업
34 A000500 1 가온전선
35 A000520 1 삼일제약
36 A000540 1 흥국화재
37 A000545 1 흥국화재우
38 A000547 1 흥국화재2우B
39 A000590 1 CS홀딩스
40 A000640 1 동아쏘시오홀딩스
41 A000650 1 천일고속
42 A000660 1 SK하이닉스
43 A000670 1 영풍
44 A000680 1 LS네트웍스
45 A000700 1 유수홀딩스
46 A000720 1 현대건설
47 A000725 1 현대건설우
48 A000760 1 이화산업
49 A000810 1 삼성화재
50 A000815 1 삼성화재우
51 A000850 1 화천기공
52 A000860 1 강남제비스코
53 A0008

827 A126560 1 현대퓨처넷
828 A128820 1 대성산업
829 A128940 1 한미약품
830 A129260 1 인터지스
831 A130660 1 한전산업
832 A130680 10 TIGER 원유선물Enhanced(H)
833 A130730 10 KOSEF 단기자금
834 A131890 10 KINDEX 삼성그룹동일가중
835 A132030 10 KODEX 골드선물(H)
836 A133690 10 TIGER 미국나스닥100
837 A133820 1 화인베스틸
838 A134380 1 미원화학
839 A134790 1 시디즈
840 A136340 10 KBSTAR 중기우량회사채
841 A136490 1 선진
842 A137610 10 TIGER 농산물선물Enhanced(H)
843 A137930 10 마이다스 200커버드콜5%OTM
844 A138040 1 메리츠금융지주
845 A138230 10 KOSEF 미국달러선물
846 A138250 1 엔에스쇼핑
847 A138490 1 코오롱플라스틱
848 A138520 10 TIGER 삼성그룹펀더멘털
849 A138530 10 TIGER LG그룹+펀더멘털
850 A138540 10 TIGER 현대차그룹+펀더멘털
851 A138910 10 KODEX 구리선물(H)
852 A138920 10 KODEX 콩선물(H)
853 A138930 1 BNK금융지주
854 A139130 1 DGB금융지주
855 A139220 10 TIGER 200 건설
856 A139230 10 TIGER 200 중공업
857 A139240 10 TIGER 200 철강소재
858 A139250 10 TIGER 200 에너지화학
859 A139260 10 TIGER 200 IT
860 A139270 10 TIGER 200 금융
861 A139280 10 TIGER 경기방어
862 A139290 10 TIGER 200 경기소비재
863 A139310 10 TIGER 금속선물(H)
864 A139320 10 TIGER 금은선물(H)
8

In [4]:
stockitems = pd.read_csv('stockitems.csv')
len(stockitems)

3072

In [4]:
import win32com.client

instCpCodeMgr = win32com.client.Dispatch("CpDib.StockCur")
#codeList = instCpCodeMgr.GetStockListByMarket()

instCpCodeMgr.SetInputValue(1)

instCpCodeMgr.BlockRequest()
num = instCpCodeMgr.GetHeaderValue(0)
print(num)

print(instCpCodeMgr.GetDataValue(0,0))


com_error: (-2147221005, '잘못된 클래스 문자열입니다.', None, None)

In [2]:
import pandas as pd
dailychart_prev = pd.read_csv('dailychart_index.csv')

In [3]:
dailychart_prev 

,code,section,date,open,high,low,close
0,U001,KOSPI,20140102,2013.109985,2013.890015,1967.189941,1967.189941
1,U001,KOSPI,20140103,1963.719971,1964.630005,1936.150024,1946.140015
2,U001,KOSPI,20140106,1947.619995,1961.849976,1943.739990,1953.280029
3,U001,KOSPI,20140107,1947.650024,1965.739990,1947.079956,1959.439941
4,U001,KOSPI,20140108,1965.500000,1966.949951,1950.020020,1958.959961
...,...,...,...,...,...,...,...
3587,U201,KOSDAQ,20210416,1015.710022,1022.359985,1014.400024,1021.619995
3588,U201,KOSDAQ,20210419,1022.059998,1032.640015,1020.580017,1029.459961
3589,U201,KOSDAQ,20210420,1029.180054,1031.900024,1025.479980,1031.880005
3590,U201,KOSDAQ,20210421,1031.260010,1031.260010,1019.590027,1022.219971


In [1]:
import win32com.client

# Create object
instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

# SetInputValue
instStockChart.SetInputValue(0, "A003540")
instStockChart.SetInputValue(1, ord('2'))
instStockChart.SetInputValue(4, 10)
instStockChart.SetInputValue(5, (0, 2, 3, 4, 5, 8))
instStockChart.SetInputValue(6, ord('D'))
instStockChart.SetInputValue(9, ord('1'))

# BlockRequest
instStockChart.BlockRequest()

# GetHeaderValue
numData = instStockChart.GetHeaderValue(3)
numField = instStockChart.GetHeaderValue(1)

# GetDataValue
for i in range(numData):
    for j in range(numField):
        print(instStockChart.GetDataValue(j, i), end=" ")
    print("")

20210726 18650 18700 18600 18650 14245 
20210723 18550 18700 18450 18600 40213 
20210722 18450 18650 18450 18550 63654 
20210721 18600 18700 18350 18400 73452 
20210720 18350 18550 18300 18500 113086 
20210719 18800 18950 18550 18650 68925 
20210716 18600 19000 18550 18950 71363 
20210715 18700 18700 18500 18600 93696 
20210714 18450 18750 18400 18650 59451 
20210713 18750 18900 18450 18600 136122 


In [6]:
import pandas as pd

dailychart = pd.read_csv('dailychart_index.csv')

dailychart.columns.tolist()

a = ['code', 'section', 'open', 'high', 'low', 'close', 'date']

dailychart.columns.tolist() == a

False